In [2]:
import os
import torch
import numpy as np
import pytorch_lightning as pl
from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from lightning.pytorch import loggers as pl_loggers
import torch_geometric.transforms as T
import torch_geometric.data as geom_data
from torch.utils.tensorboard import SummaryWriter

%reload_ext autoreload
%autoreload 2

#### Configure device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != "cpu":
    torch.set_float32_matmul_precision('high')

num_workers = os.cpu_count()
device, num_workers

(device(type='cuda', index=0), 12)

### Directory configuration and load data

In [4]:
cwd = os.getcwd()
tb_logging_dir = os.path.join(cwd, "lightning_logs")
exp_name = "Cora-GCN"
exp_dir = os.path.join(tb_logging_dir, exp_name)
dataset_dir = os.path.join(cwd, "utils/dataset", "Cora")
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
CoraDataset = Planetoid(
    root="dataset/Cora", name="Cora", split="full"
)

#### Note that if numebr of epoch is large, the kernel will stuck after training! Need to load the saved model mannually!

In [5]:
from utils.model.RS_GCN import GCN

dataset = CoraDataset

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00,
    patience=10,
    verbose=False,
    mode='min'
)
hparams = {"DROUPOUT_RATE": 0,
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }  # The number of neighbors in each order of sampling

tb_logger = pl_loggers.TensorBoardLogger(tb_logging_dir, name=exp_name)

trainer = pl.Trainer(max_epochs=100,
                     callbacks=[early_stop_callback],
                     logger=tb_logger,
                     log_every_n_steps=1,
                     )

version_dir = os.path.join(
    exp_dir, "version_"+str(trainer.logger.version))
writer_acc = SummaryWriter(log_dir=version_dir)
writer_loss = SummaryWriter(log_dir=version_dir)

checkpoint_dir = os.path.join(version_dir, "checkpoints")
print("Saving checkpoints to", checkpoint_dir)

GCNmodel = GCN(dataset=dataset, input_dim=dataset.num_features, hparams=hparams,
               writer_acc=writer_acc, writer_loss=writer_loss).to(device)
GCNmodel.data_processing()

trainer.fit(GCNmodel)
trainer.test(GCNmodel)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Saving checkpoints to D:\GitHub\aml-project\GNN\zyq\lightning_logs\Cora-GCN\version_15\checkpoints
Data Processing Done on: cuda:0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | conv1 | GCNConv | 160 K 
1 | conv2 | GCNConv | 1.8 K 
----------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.650     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9716257487539594     │
│         test_loss         │    0.5124006271362305     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5124006271362305, 'test_acc': 0.9716257487539594}]

#### Upload trained model

In [ ]:
from utils.model.RS_GCN import GCN

hparams = {"DROUPOUT_RATE": 0,
           "BATCH_SIZE": 64,
           "LEARNING_RATE": 0.001,
           "NUM_NEIGHBORS": [10, 10]
           }  # The number of neighbors in each order of sampling

dataset = CoraDataset
checkpoint_dir = "lightning_logs\\RS_Cora_GCN\\best\\checkpoints"
checkpoint_file = os.path.join(checkpoint_dir, os.listdir(checkpoint_dir)[0])
Loaded_model = GCN.load_from_checkpoint(
    checkpoint_file, dataset=dataset, input_dim=dataset.num_features, hparams=hparams
)
Loaded_model.data_processing()

#### To see the mini-batch's number of negative edges:

In [6]:
tmpLoader = Loaded_model.train_dataloader()
sample = next(iter(tmpLoader))
sample

Data(x=[2474, 1433], edge_index=[2, 6290], edge_label=[6290], train_mask=[2474], n_id=[2474], e_id=[6290], input_id=[64], batch_size=64)

#### To see the number of training positive edges:

In [7]:
Loaded_model.pos_data["train"]

Data(x=[2708, 1433], edge_index=[2, 4708], edge_label=[4708], train_mask=[2708])

#### Recommendation for a given node

In [8]:
import torch.nn.functional as F
def recommendation(A, model):
    ## Return recommedation for A in the given graph
    neg_graph = model.neg_data["all"]
    candidate = torch.zeros(neg_graph.num_edges)

    candidate = neg_graph.edge_index[1, (neg_graph.edge_index[0] == A)].cpu()
    candidate.sort()
    num_candidates = len(candidate) 
    u = torch.ones(num_candidates, dtype=torch.int) * A
    v = torch.Tensor(candidate)
    candidate_link = torch.stack((u, v))

    embedding = model(model.pos_data["all"].x, model.pos_data["all"].edge_index)

    scores = np.zeros(candidate.max()+1) - 1e8
    
    scores[v] = F.sigmoid((embedding[u] * embedding[v]).sum(dim=1)).detach().numpy()

    rank_idx = np.argsort(scores)[::-1]
    print("Recommendation for node", A)
    for k in range(10):
        print(rank_idx[k], scores[rank_idx[k]])
        
recommendation(1676, Loaded_model)

Recommendation for node 1676
1224 0.9231213927268982
2394 0.9167168140411377
1973 0.9015019536018372
1402 0.8846403360366821
1421 0.8812358379364014
779 0.8793782591819763
1464 0.8793158531188965
1919 0.8666505813598633
1370 0.8663382530212402
175 0.8641014695167542
